### Домашнее задание

1. Для нашего пайплайна (Case1) поэкспериментировать с разными моделями: 1 - бустинг, 2 - логистическая регрессия (не забудьте здесь добавить в cont_transformer стандартизацию - нормирование вещественных признаков)
2. Отобрать лучшую модель по метрикам (кстати, какая по вашему мнению здесь наиболее подходящая DS-метрика)
3. Для отобранной модели (на отложенной выборке) сделать оценку экономической эффективности при тех же вводных, как в вопросе 2 (1 доллар на привлечение, 2 доллара - с каждого правильно классифицированного (True Positive) удержанного). (подсказка) нужно посчитать FP/TP/FN/TN для выбранного оптимального порога вероятности и посчитать выручку и траты. 
4. (опционально) Провести подбор гиперпараметров лучшей модели по итогам 2-3
5. (опционально) Еще раз провести оценку экономической эффективности

### Практика

### Case 1

Давайте поработаем с набором данных с платформы kaggle https://www.kaggle.com/adammaus/predicting-churn-for-bank-customers по оттоку клиентов банка

In [1]:
import itertools
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline, make_pipeline, Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix, average_precision_score


%matplotlib inline

In [2]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in self.columns:
            if col_ not in test_columns:
                X[col_] = 0
        return X[self.columns]

In [3]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
      

def get_metrics(y_test, probs, money_to_keep_one_client=1, expected_profit_from_one_client_cash_eq=2, currency='dollars', fstr=True):
    """
    Функция перехода от вероятностей к меткам классов.
    Для этого нужно подобрать порог - Best_Threshold={thresholds[ix]:.3f},
    после которого мы считаем,
    что объект можно отнести к классу 1 
    (если вероятность больше порога -
    размечаем объект как класс 1,
    если нет - класс 0)

    Args:
        y_test ([type]): [Истинные классы]
        probs ([type]): [Предсказанные вероятности принадлежности к классу]
        money_to_keep_one_client (int, optional): [Затраты на удержание одного клиента]. Defaults to 1.
        expected_profit_from_one_client_cash_eq (int, optional): [потенциальная выгода от одного целевого клиента]. Defaults to 2.
        currency (str, optional): [валюта измерения]. Defaults to 'dollars'.
        fstr (bool, optional): [флаг вывода]. Defaults to True.

    Returns:
        if fstr is True:
            [f'str']: [Выводиться f-string в виде: 
                        f'Best_Threshold={thresholds[ix]:.3f},\n'
                        f'F_Score={fscore[ix]:.3f},\n'
                        f'Precision={precision[ix]:.3f},\n'
                        f'Recall={recall[ix]:.3f},\n'
                        f'Roc_AUC={roc_auc_score(y_test, probs)}'
                        f'Potential_economic_benefit = {potential_economic_benefit} {currency}']
        else:
            [tuple]: [(
                       thresholds[ix]: float,
                       fscore[ix]: float,
                       precision[ix]: float,
                       recall[ix]: float,
                       roc_auc_score(y_test, probs): float,
                       potential_economic_benefit[потенциально экономическая целесобразность в денежном эквиваленте]: float 
                       )]
    """
    precision, recall, thresholds = precision_recall_curve(y_test, probs)

    fscore = (2 * precision * recall) / (precision + recall)

    ix = np.argmax(fscore)
    cnf_matrix = confusion_matrix(y_test, probs>thresholds[ix])
    tn, fp, fn, tp = cnf_matrix.ravel()
    sum_expected_profit_from_clients_cash_eq = (tp)*expected_profit_from_one_client_cash_eq
    sum_money_to_keep_clients = (fp+tp)*money_to_keep_one_client
    potential_economic_benefit = sum_expected_profit_from_clients_cash_eq - sum_money_to_keep_clients
    if fstr:
        return(f'Best_Threshold={thresholds[ix]:.3f},\n'
               f'F_Score={fscore[ix]:.3f},\n'
               f'Precision={precision[ix]:.3f},\n'
               f'Recall={recall[ix]:.3f},\n'
               f'Roc_AUC={roc_auc_score(y_test, probs)},\n'
               f'Potential_economic_benefit = {potential_economic_benefit} {currency}')
    else:
        return thresholds[ix], fscore[ix], precision[ix], recall[ix], roc_auc_score(y_test, probs), potential_economic_benefit

In [4]:
df = pd.read_csv("churn_data.csv")
df.head(3)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


Есть как категориальные, так и вещественные признаки. Поле CustomerId нужно будет удалить. 

Посмотрим на распределение классов:

In [5]:
df['Exited'].value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

Не самое плохое распределение (1 к 4)

Давайте построим модель. Сразу же будем работать с использованием sklearn pipeline

In [6]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(df, df['Exited'], random_state=21)

In [7]:
df.head(3)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


Зададим списки признаков

In [8]:
categorical_columns = ['Geography', 'Gender', 'Tenure', 'HasCrCard', 'IsActiveMember']
continuous_columns = ['CreditScore', 'Age', 'Balance', 'NumOfProducts', 'EstimatedSalary']

- Категориальные признаки закодируем с помощью OneHotEncoding
- Вещественные отмаштабируем MinMaxScaler((-1, 1))

Теперь нам нужно под каждый признак создать трансформер и объединить их в список (сделаем это в цикле, чтобы не мучиться)

In [9]:
final_transformers = list()

for cat_col in categorical_columns:
    cat_transformer = Pipeline([
                ('selector', FeatureSelector(column=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    final_transformers.append((cat_col, cat_transformer))
    
for cont_col in continuous_columns:
    cont_transformer = Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', MinMaxScaler((-1, 1)))
            ])
    final_transformers.append((cont_col, cont_transformer))

Объединим все это в единый пайплайн

In [10]:
feats = FeatureUnion(final_transformers)

feature_processing = Pipeline([('feats', feats)])

Теперь у нас есть пайплайн, который готовит признаки для моделирования.

### Добавим модели

In [11]:
pipeline_rfc = Pipeline([
    ('features',feats),
    ('classifier', RandomForestClassifier(max_depth=10,
                                          max_features=0.7, 
                                          min_samples_leaf=6,
                                          random_state=21)),
])


pipeline_lr = Pipeline([
    ('features', feats),
    ('classifier', LogisticRegression(random_state = 21)),
])

params_est = {
    'n_estimators': 300,
    'loss': 'exponential',
    'learning_rate': 0.08,
    'subsample': 0.6910000000000001,
    'min_samples_leaf': 3,
    'max_features': 10,
    'random_state': 21,
    'verbose': 1
}
pipeline_gbc = Pipeline([
    ('features', feats),
    ('classifier', GradientBoostingClassifier(**params_est)),
])


params = {
    'silent': 1,
    'objective': 'binary:logistic',
    'max_depth': 4,
    'eta': 0.01,
    'subsample': 0.4,
    'min_child_weight': 7,
    'n': 580,
    'verbose': 1
}

pipeline_xgb = Pipeline([
    ('features', feats),
    ('classifier', xgb.XGBClassifier(boosting_type='gbdt', **params)),
])


### Обучим модели на пайплайне

In [12]:
pipeline_rfc.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Geography',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Geography')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Geography'))])),
                                                ('Gender',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Gender')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Gender'))])),
                                                ('Tenure',
                                                 Pipeline(steps=[('selector',
           

In [13]:
preds_rfc = pipeline_rfc.predict_proba(X_test)[:, 1]
preds_rfc[:10]

array([0.00637401, 0.01128901, 0.01085011, 0.00539659, 0.00547304,
       0.78429434, 0.68264285, 0.00642244, 0.43459274, 0.04325599])

In [14]:
print(get_metrics(y_test, preds_rfc))

Best_Threshold=0.328,
F_Score=0.638,
Precision=0.652,
Recall=0.625,
Roc_AUC=0.8644041986292756,
Potential_economic_benefit = 148 dollars


In [15]:
pipeline_lr.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Geography',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Geography')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Geography'))])),
                                                ('Gender',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Gender')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Gender'))])),
                                                ('Tenure',
                                                 Pipeline(steps=[('selector',
           

In [16]:
preds_lr = pipeline_lr.predict_proba(X_test)[:, 1]
preds_lr[:10]

array([0.06375403, 0.14155555, 0.11618981, 0.02720937, 0.03729373,
       0.52771899, 0.36045306, 0.06220476, 0.25224518, 0.025757  ])

In [17]:
print(get_metrics(y_test, preds_lr))

Best_Threshold=0.815,
F_Score=nan,
Precision=0.000,
Recall=0.000,
Roc_AUC=0.7625892071177062,
Potential_economic_benefit = -4 dollars


<ipython-input-3-78ae28e9838d>:74: RuntimeWarning: invalid value encountered in true_divide
  fscore = (2 * precision * recall) / (precision + recall)


In [18]:
pipeline_gbc.fit(X_train, y_train)

      Iter       Train Loss      OOB Improve   Remaining Time 
         1           0.7907           0.0132            2.99s
         2           0.7772           0.0132            2.24s
         3           0.7664           0.0074            1.49s
         4           0.7636           0.0090            1.48s
         5           0.7467           0.0069            1.77s
         6           0.7455           0.0090            1.47s
         7           0.7320           0.0067            1.46s
         8           0.7318           0.0103            1.46s
         9           0.7219           0.0066            1.46s
        10           0.7107           0.0081            1.45s
        20           0.6753           0.0029            1.37s
        30           0.6226           0.0017            1.29s
        40           0.6173           0.0021            1.23s
        50           0.5954           0.0007            1.17s
        60           0.5955           0.0005            1.14s
       

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Geography',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Geography')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Geography'))])),
                                                ('Gender',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Gender')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Gender'))])),
                                                ('Tenure',
                                                 Pipeline(steps=[('selector',
           

In [19]:
preds_gbc = pipeline_gbc.predict_proba(X_test)[:, 1]
preds_gbc[:10]

array([0.0195897 , 0.01747412, 0.02350595, 0.0045466 , 0.00608024,
       0.7332305 , 0.35476227, 0.00722138, 0.48018977, 0.02437262])

In [20]:
print(get_metrics(y_test, preds_gbc))

Best_Threshold=0.294,
F_Score=0.662,
Precision=0.642,
Recall=0.684,
Roc_AUC=0.8734781737298793,
Potential_economic_benefit = 154 dollars


In [21]:
pipeline_xgb.fit(X_train, y_train)

[00:32:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "boosting_type", "n", "silent", "verbose" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[00:32:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


D:\programming\anaconda3\lib\site-packages\xgboost-1.4.1-py3.8-win-amd64.egg\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Geography',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Geography')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Geography'))])),
                                                ('Gender',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Gender')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Gender'))])),
                                                ('Tenure',
                                                 Pipeline(steps=[('selector',
           

### наши прогнозы для тестовой выборки

In [22]:
preds_xgb = pipeline_xgb.predict_proba(X_test)[:, 1]
preds_xgb[:10]

array([0.20332171, 0.2049284 , 0.20462176, 0.20049337, 0.20072055,
       0.6066431 , 0.46400535, 0.20135792, 0.42206845, 0.24893394],
      dtype=float32)

In [23]:
print(get_metrics(y_test, preds_xgb))

Best_Threshold=0.373,
F_Score=0.614,
Precision=0.605,
Recall=0.623,
Roc_AUC=0.8540068536217303,
Potential_economic_benefit = 110 dollars
